In [17]:
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

# Finding Giant Component

In [18]:
from pathlib import Path

path = Path('../data/processed_except/pantip_edge_10.csv').resolve();
di_graph = nx.DiGraph()
with open(path, 'r') as input_file:
    while True:
        line = input_file.readline().rstrip()

        # EOF
        if line == '':
            break

        splits = line.split(',')
        # prevent self-loop
        if splits[0] != splits[1]:
            di_graph.add_edge(splits[1], splits[0])

# Finding the Giant Component
undi_graph = di_graph.to_undirected()
graph_components = (undi_graph.subgraph(c) for c in nx.connected_components(undi_graph));
graph_components = sorted(graph_components, key=len, reverse=True)

giant_component =  di_graph.subgraph(graph_components[0].nodes())

edge_df = pd.DataFrame(columns=[ 'Source','Destination' ])
for index, edge in enumerate(giant_component.edges()):
    (src, dest) = edge
    edge_df.loc[index] = [ src, dest ]

edge_df.head()

,Source,Destination
0,30946662,35274197
1,30946662,30137128
2,36213351,35877063
3,36213351,34731825
4,36213351,35841112


# PageRank

In [19]:
pr = nx.pagerank(giant_component)
pr_tuple = []
for p in pr:
    pr_tuple.append((p, pr[p]))
pr_tuple = sorted(pr_tuple, key=lambda x: x[1], reverse=True)
pr_df = pd.DataFrame(columns=['TopicID', 'PageRank Value'])
for index, prt in enumerate(pr_tuple):
    pr_df.loc[index] = [prt[0], prt[1]]
pr_df.head(10)

,TopicID,PageRank Value
0,38176352,0.002464
1,34502095,0.002418
2,34222014,0.002349
3,32983135,0.002219
4,32447263,0.002058
5,38179329,0.002020
6,34433699,0.001966
7,36111851,0.001948
8,35268526,0.001925
9,31791000,0.001850


#### Finding Topic-Tags

In [20]:
from pathlib import Path
import json
topic_tags = {}
tag_path = Path('../data/processed_except/topic_tags.json').resolve();
with open(tag_path, 'r', encoding='utf-8') as tf:
    while True:
        line = tf.readline()
        if line == '':
            break
        cur_json = json.loads(line)
        topic_tags[cur_json['topic_id']] = cur_json['tags']

#### PageRank Result

In [21]:
from pathlib import Path
import json
topics_title = {}
path = Path('../data/processed_except/topics.json').resolve();
with open(path, 'r', encoding='utf-8') as tf:
    while True:
        line = tf.readline()
        if line == '':
            break
        cur_json = json.loads(line)
        topics_title[cur_json['topic_id']] = cur_json['title']

In [45]:
RANK_LIMIT = 10
count = 0
index = 0
recommend_set = set()
rank_df = pd.DataFrame(columns=['TopicID', 'Topic Title', 'Topic Tags', 'PageRank Value'])
for i in range(1000):
    if count >= RANK_LIMIT:
        break
    cur_set = set(topic_tags[pr_tuple[i][0]])
    common_set = recommend_set.intersection(cur_set)
#     if len(common_set) < len(recommend_set) * 0.5 or len(recommend_set) == 0:
    if len(common_set) == 0:
        for x in cur_set:
            recommend_set.add(x)
        index = i
        rank_df.loc[count] = [pr_tuple[i][0], topics_title[pr_tuple[i][0]], topic_tags[pr_tuple[i][0]], pr_tuple[i][1]]
        count += 1
rank_df.head(RANK_LIMIT)

,TopicID,Topic Title,Topic Tags,PageRank Value
0,38176352,TG971 ดีเลย์ 2 ชั่วโมงครึ่ง เหตุนักบินไม่ยอมเท...,"[สายการบิน, การบินไทย]",0.002464
1,34502095,[CR] ตะลุยกิน15ร้านเด็ด Osaka-Kyoto-Kobe-Nara ...,"[เที่ยวต่างประเทศ, อาหารญี่ปุ่น, ภาพถ่าย, คันไ...",0.002418
2,32983135,[CR] @@ รีวิว เที่ยวใกล้บ้าน ... นครนายก พักที...,"[เที่ยวไทย, บันทึกนักเดินทาง]",0.002219
3,31791000,[CR] รีวิว : พัทยา 2 วัน 1 คืน (วันเสาร์ที่ผ่า...,"[โรงแรมรีสอร์ท, เที่ยวทะเล, พัทยา]",0.001850
4,31170821,หนาวขนาดไหน แต่งตัวยังไงให้พอดี มีคำตอบ,"[เรียนต่อต่างประเทศ, แม่บ้านต่างแดน, ทำงานต่าง...",0.001696
5,38172159,ตม.เกาหลีรวบผีน้อยไทย 9 ราย ขณะนั่งกินหมูกระทะ,"[เที่ยวเกาหลี, เจ้าหน้าที่ตรวจคนเข้าเมือง, ราย...",0.001382
6,38172968,แชร์ประสบการณ์ขนลุก ขอวีซ่าเชงเก้นอิตาลีไม่ผ่า...,"[วีซ่า, Tourist Visa, Schengen Visa]",0.001165
7,36513654,[CR] รีวิว เจ็ดคต-โป่งก้อนเส้า กางเต็นท์ พักผ่...,"[อาหารไทย, เดินป่า, เที่ยวเชิงอนุรักษ์, อาหาร,...",0.001001
8,32844106,### ไปสิงคโปร์ ไปซื้ออะไรดี หนูคิสสิบอกให้เด้อ...,"[แต่งหน้า, เครื่องสำอาง, Skin Care, คนไทยในสิง...",0.000948
9,34111777,[CR] รีวิว 10ร้าน 10สไตล์... คาเฟ่ไหนน่าไปเช็ค...,"[ของหวาน, ร้านอาหาร, ภาพถ่ายจากกล้องโทรศัพท์, ...",0.000875


# HITS

In [41]:
hubs, auths = nx.hits(giant_component, max_iter=500)
hits_data = {}
for hub in hubs:
    hits_data[hub] = [ hubs[hub] ]
for auth in auths:
    hits_data[auth].append(auths[auth])

hits_data_arr = []
for index, topic in enumerate(hits_data):
    cur_data = hits_data[topic]
    hits_data_arr.append([ topic, cur_data[0], cur_data[1] ])

hits_data_arr = sorted(hits_data_arr, key=lambda x: x[2], reverse=True)

hits_df = pd.DataFrame(columns=['TopicID', 'Topic Title', 'Hub', 'Authority'])
for index, data in enumerate(hits_data_arr):
    hits_df.loc[index] = [ data[0], topics_title[data[0]], data[1], data[2] ]

In [46]:
hits_df.head(10)

,TopicID,Topic Title,Hub,Authority
0,38122354,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 5 แหล่งบั...,0.014925,0.115598
1,38116097,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 4 รากฐานช...,0.019620,0.099313
2,38104881,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 1 ปฐมบทจุ...,0.089684,0.093230
3,38128544,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 6 แหล่งบั...,0.036114,0.087938
4,38106313,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 2 โรงแรมน...,0.045879,0.078350
5,38111299,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 3 เมื่อฉั...,0.049774,0.077767
6,38140936,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 7 ประเทศน...,0.108599,0.075499
7,38178317,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 11 ปัจฉิม...,0.094059,0.069434
8,38147148,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 8 เกาหลีเ...,0.136068,0.065974
9,38173055,[CR] เกาหลีเหนือในความทรงจำ | ตอนที่ 10 เมืองห...,0.098851,0.055281
